# BAX-422 Project 1
-- Anakin Liu, Jingyu Tang, Ruijie Shan, Ruiyuan Yang

<br/>

## Smart Travel Planner

#### APIs used:
* **OpenWhether** - https://openweathermap.org/api
* **Amadeus** - https://developers.amadeus.com/self-service

<br/>

<br/>

### Libraries

In [1]:
import os
import requests as rq
from datetime import datetime

<br/>

### API Key Extract Function

In [2]:
def ext_api(file_name: str)->dict[str, str]:
    api_dict = {}
    with open(file_name, 'r') as file:
        for line in file:
            key, value = line.strip().split('=', 1)
            api_dict[key.strip()] = value.strip()
    return api_dict

<br/>

### Functionalities Class

In [3]:
class SmartTravelPlanner:
    def __init__(self, weather_key:str, amadeus_key_pack:tuple[str, str]):
        # Extract API keys
        self.__weather_key = weather_key

        # Base URLs for the APIs
        self.weather_base_url = "https://api.openweathermap.org/data/2.5/weather"
        self.amadeus_auth_url = "https://test.api.amadeus.com/v1/security/oauth2/token"
        self.amadeus_flight_url = "https://test.api.amadeus.com/v2/shopping/flight-offers"

        # Initialize Amadeus access token
        payload = {
            "grant_type":"client_credentials",
            "client_id":amadeus_key_pack[0],
            "client_secret":amadeus_key_pack[1]
        }
        headers = {"Content-Type": "application/x-www-form-urlencoded"}
        response = rq.post(self.amadeus_auth_url, data=payload, headers=headers)
        self.__amadeus_token = response.json().get("access_token")
    

    # Generic method for making API calls and handling errors.
    def make_api_request(self, url:str, params:dict[str, str], headers=None)->dict|None:
        try:
            response = rq.get(url, params=params, headers=headers)
            return response.json()
        except rq.exceptions.RequestException as e:
            print(f"API request failed: {e}")
            return None
        
    
    # Method for timestamp formation
    def format_timestamp(self, timestamp:int, is_24h:bool)->str:
        dt_object = datetime.fromtimestamp(timestamp)
        if is_24h:
            return dt_object.strftime("%H:%M")
        else:
            return dt_object.strftime("%I:%M %p")
    

    # Fetch weather data using OpenWeather API
    def get_weather(self, city:str, pref:str='global')->dict[str, str]:
        # Distinguish user preference
        settings = {
            "global":{"metric":"metric", "temp_unit":"C", "dist_unit":"km", "spd_unit":"m/s", "is_24h":True},
            "us":{"metric":"imperial", "temp_unit":"F", "dist_unit":"mile", "spd_unit":"mph", "is_24h":False}
        }
        rule = settings[pref]

        # API call
        params = {"q":city, "appid":self.__weather_key, "units":rule["metric"]}
        json_data = self.make_api_request(self.weather_base_url, params)

        # Convert visibility
        visibility = int(json_data['visibility'] * (0.001 if pref == 'global' else 0.000621371))

        # Convert wind direction
        wind_deg = json_data['wind']['deg']
        wind_directions = ["North", "North-East", "East", "South-East", "South", "South-West", "West", "North-West"]
        wind_dir = wind_directions[round(wind_deg / 45) % 8]

        # Formatting data for presentation
        return {
            "city": city,
            "time": self.format_timestamp(json_data['dt'], rule['is_24h']),
            "temperature": f"{json_data['main']['temp']}°{rule['temp_unit']}",
            "min_temperature": f"{json_data['main']['temp_min']}°{rule['temp_unit']}",
            "max_temperature": f"{json_data['main']['temp_max']}°{rule['temp_unit']}",
            "feels": f"{json_data['main']['feels_like']}°{rule['temp_unit']}",
            "humidity": f"{json_data['main']['humidity']}%",
            "visibility": f"{visibility} {rule['dist_unit']}",
            "weather": json_data['weather'][0]['main'],
            "weather_icon": f"http://openweathermap.org/img/wn/{json_data['weather'][0]['icon']}.png",
            "wind_speed": f"{json_data['wind']['speed']} {rule['spd_unit']}",
            "wind_dir": wind_dir,
            "sunrise": self.format_timestamp(json_data['sys']['sunrise'], rule['is_24h']),
            "sunset": self.format_timestamp(json_data['sys']['sunset'], rule['is_24h'])
        }
    

    # Fetch flight prices from Amadeus API
    def get_flight_offers(
            self, 
            origin:str, 
            destination:str, 
            departure_date:str, 
            adults:int=1, 
            max_results:int=10
        )->list[dict[str, str]]:
        # Error check
        if not self.__amadeus_token:
            return {"error": "Failed to authenticate with Amadeus API"}

        # Parameter setting
        headers = {"Authorization":f"Bearer {self.__amadeus_token}"}
        params = {
            "originLocationCode":origin,
            "destinationLocationCode":destination,
            "departureDate":departure_date,
            "adults":adults,
            "max": max_results
        }

        # API call
        json_data = self.make_api_request(self.amadeus_flight_url, params, headers)

        # Extract important information
        flight_offers = []
        for flight in json_data["data"]:
            # Airline and price information
            airline_code = flight["validatingAirlineCodes"][0]
            price = float(flight["price"]['grandTotal'])  # Convert price to float for sorting
            price_unit = flight["price"]['currency']
            
            # Flight itinerary details
            itinerary = flight["itineraries"][0]
            segments = itinerary["segments"]
            # Extract departure and arrival times
            first_segment = segments[0]
            last_segment = segments[-1]
            departure_time = first_segment["departure"]["at"]
            arrival_time = last_segment["arrival"]["at"]
            # Check if the flight is direct or requires transits
            is_direct = len(segments) == 1
            transit_info = 'Direct flight' if is_direct else 'Need to transfer'

            # Append formatted flight information
            flight_offers.append({
                "airline": airline_code,
                "price": price,  # Store as float for sorting
                "price_display": f"{price} {price_unit}",  # Separate display version
                "departure_time": departure_time,
                "arrival_time": arrival_time,
                "transit_info": transit_info,
                "carrier": first_segment["carrierCode"]
            })

        # Sort flights by price (ascending order)
        flight_offers.sort(key=lambda x: x["price"])

        # Clean up
        for flight in flight_offers:
            del flight["price"]

        return flight_offers

    
    # Fetch hotel options from Amadeus hotel API
    def get_car_rentals(self, location):
        '''delete this string and continue coding'''

<br/>

### Function Test - Setup

In [4]:
apis = ext_api('api_keys.txt')

stp = SmartTravelPlanner(apis['openweathermap_api'], (apis['amadeus_api'], apis['amadeus_secret']))

<br/>

### Function Test - Weather Information

In [ ]:
city_weather = stp.get_weather("San Francisco", pref='us')
print(city_weather)

{'city': 'San Francisco', 'time': '08:54 PM', 'temperature': '55.11°F', 'min_temperature': '53.06°F', 'max_temperature': '57.38°F', 'feels': '54.32°F', 'humidity': '85%', 'visibility': '6 mile', 'weather': 'Rain', 'weather_icon': 'http://openweathermap.org/img/wn/10n.png', 'wind_speed': '17.27 mph', 'wind_dir': 'South-East', 'sunrise': '07:11 AM', 'sunset': '05:35 PM'}


<br/>

### Function Test - Flight Information

In [5]:
res = stp.get_flight_offers('SFO', 'JFK', '2025-03-10')
for r in res:
    print(r)

{'airline': 'F9', 'price_display': '68.12 EUR', 'departure_time': '2025-03-10T18:18:00', 'arrival_time': '2025-03-11T08:00:00', 'transit_info': 'Need to transfer', 'carrier': 'F9'}
{'airline': 'F9', 'price_display': '73.5 EUR', 'departure_time': '2025-03-10T12:24:00', 'arrival_time': '2025-03-11T08:00:00', 'transit_info': 'Need to transfer', 'carrier': 'F9'}
{'airline': 'B6', 'price_display': '120.06 EUR', 'departure_time': '2025-03-10T05:30:00', 'arrival_time': '2025-03-10T13:54:00', 'transit_info': 'Direct flight', 'carrier': 'B6'}
{'airline': 'B6', 'price_display': '120.06 EUR', 'departure_time': '2025-03-10T14:10:00', 'arrival_time': '2025-03-10T22:38:00', 'transit_info': 'Direct flight', 'carrier': 'B6'}
{'airline': 'AS', 'price_display': '120.06 EUR', 'departure_time': '2025-03-10T13:20:00', 'arrival_time': '2025-03-10T22:01:00', 'transit_info': 'Direct flight', 'carrier': 'AS'}
{'airline': 'AS', 'price_display': '129.39 EUR', 'departure_time': '2025-03-10T08:00:00', 'arrival_tim

<br/>

### Function Test - Hotel Information